In [1]:
# Weaving cells in pidgin programs

# Weaving cells in pidgin programs

In [2]:
pidgin programming is an incremental approach to documents.

pidgin programming is an incremental approach to documents.

In [ ]:
    import datetime, dataclasses, sys, IPython as python, IPython, nbconvert as export, collections, IPython as python, mistune as markdown, hashlib
    exporter, shell = export.exporters.TemplateExporter(), python.get_ipython()
    modules = lambda:[x for x in sys.modules if '.' not in x and not str.startswith(x,'_')]

In [ ]:
    def load_ipython_extension(shell):
        global metadata
        metadata.shell = shell
        shell.events.register('pre_run_cell', metadata.pre_run_cell)
        shell.events.register('post_run_cell', metadata.post_run_cell)
        shell.display_formatter.formatters['text/markdown'].for_type(str, lambda x: x)
        

In [ ]:
Markdown input can fail to render when jinja2 is used in correctly.  Markdown is never wrong, but sometimes jinja is.

In [ ]:
    import hashlib

In [ ]:
    @dataclasses.dataclass
    class Metadata:
        def pre_run_cell(self, info):
            self.modules = modules()
            self.start = datetime.datetime.utcnow().isoformat()
            if hasattr(self.shell, 'user_ns'):
                self.ns = self.names()
            
            return info
        
        def names(self):
            return [x for x in self.shell.user_ns if x[0].islower()]

        def post_run_cell(self, result):
            text = result.info.raw_cell
            lines = text.splitlines() or ['']
            if lines[0].strip(): 
                exporter.environment.filters.update({
                    k: v for k, v in getattr(self.shell, 'user_ns', {}).items() if callable(v) and k not in exporter.environment.filters})
                try:
                    text = exporter.environment.from_string(text, globals=getattr(self.shell, 'user_ns', {})).render()
                except BaseException as Exception:
                    self.shell.showtraceback((type(Exception), Exception, Exception.__traceback__))
                    
                IPython.display.display(IPython.display.Markdown(text, metadata=dict(
                    modules=[x for x in modules() if x not in self.modules], 
                    names=[x for x in self.names() if x not in self.ns],
                    start_time=self.start,
                    end_time=datetime.datetime.utcnow().isoformat(),
                    #id=hashlib.sha256(bytes(id(self.shell))).hexdigest(),
                )))
            return result
        
        def __post_init__(self):
            self.ns = [x for x in getattr(self.shell, 'user_ns', {}) if '.' not in x and not str.startswith(x,'_')]
        shell: python.InteractiveShell = dataclasses.field(default_factory=python.get_ipython)
        start: datetime.datetime = dataclasses.field(default_factory=datetime.datetime.utcnow().isoformat)
        modules: list = dataclasses.field(default_factory=list)
        ns: list = dataclasses.field(init=False)

In [ ]:
    metadata = Metadata()

In [ ]:
    
    def unload_ipython_extension(shell):
        global metadata
        metadata.shell = shell
        try:
            shell.events.unregister('pre_run_cell', metadata.pre_run_cell)
            shell.events.unregister('post_run_cell', metadata.post_run_cell)
        except:...